In [ ]:
// Pastikan 'lembang2' sudah didefinisikan sebagai ee.FeatureCollection atau ee.Geometry
// Contoh:
// var lembang2 = ee.FeatureCollection('users/YOUR_USERNAME/YOUR_ASSET_NAME');
// Map.centerObject(lembang2, 10);
// Map.addLayer(lembang2, {}, 'Lembang Boundary');

// --- Definisi Dataset ---
// Menggunakan CHIRPS Daily: Climate Hazards Group InfraRed Precipitation with Station data (Final)
// Kita akan melakukan agregasi bulanan dari data harian.
var CHIRPS = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY");

// --- Rentang Waktu dan Parameter ---
var startYear = 2013;
var startMonth = 1; // CHIRPS tersedia dari awal tahun 2013
var endYear = 2022;
var endMonth = 12; // Desember
var numSamples = 1000; // Jumlah titik sampel acak per bulan
var scaleForSampling = 30; // Resolusi untuk sampling (akan meresampling CHIRPS ke 30m)

// --- Fungsi untuk Memproses Data CHIRPS Bulanan ---
var processMonthlyCHIRPS = function(date) {
  date = ee.Date(date);
  var endDate = date.advance(1, 'month');

  // Filter CHIRPS untuk bulan ini
  var CHIRPS_monthly_raw = CHIRPS.filterBounds(lembang2)
                                 .filterDate(date, endDate);

  // Hitung total curah hujan bulanan (sum)
  // Band default di CHIRPS adalah 'precipitation'
  var monthly_precipitation = CHIRPS_monthly_raw.select('precipitation').sum().clip(lembang2);

  // **Penting:** Periksa apakah citra sum memiliki band 'precipitation'
  var hasCHIRPSData = monthly_precipitation.bandNames().contains('precipitation');

  if (!hasCHIRPSData) {
    // Debugging: uncomment baris di bawah untuk melihat bulan mana yang kosong
    // print('No valid CHIRPS data for ' + date.format('YYYY-MM'));
    return ee.FeatureCollection([]); // Kembalikan koleksi kosong jika tidak ada data
  }

  // Ganti nama band agar lebih jelas
  var CHIRPS_final = monthly_precipitation.rename('CHIRPS_mm');

  // --- Sampling Acak ---
  var samplePoints = CHIRPS_final.sample({
    region: lembang2,
    scale: scaleForSampling, // GEE akan meresampling CHIRPS ke resolusi ini
    numPixels: numSamples,
    geometries: true,
    seed: 0,
    tileScale: 16
  });

  // Tambahkan properti waktu ke setiap fitur sampel
  samplePoints = samplePoints.map(function(feature) {
    return feature.set({
      'year': date.get('year'),
      'month': date.get('month'),
      'date': date.format('YYYY-MM-dd')
    });
  });

  // Debugging: uncomment baris di bawah untuk melihat jumlah sampel per bulan
  // print('Samples for ' + date.format('YYYY-MM') + ':', samplePoints.size());

  return samplePoints;
};

// --- Buat daftar semua bulan dalam rentang waktu ---
var dates = [];
for (var y = startYear; y <= endYear; y++) {
  var currentMonth = (y === startYear) ? startMonth : 1;
  var lastMonth = (y === endYear) ? endMonth : 12;

  for (var m = currentMonth; m <= lastMonth; m++) {
    dates.push(ee.Date.fromYMD(y, m, 1));
  }
}

// Proses data untuk setiap bulan
var all_sampled_CHIRPS_data = ee.FeatureCollection(ee.List(dates).map(processMonthlyCHIRPS)).flatten();

// --- Ekspor Data ke Google Drive ---
Export.table.toDrive({
  collection: all_sampled_CHIRPS_data,
  description: 'CHIRPS_Monthly_Lembang_Samples',
  folder: 'GEE_Kekeringan_Data',
  fileFormat: 'CSV'
});

// --- Opsional: Visualisasi satu contoh bulan (Misal, Juli 2013) ---
/*
var testDate = ee.Date.fromYMD(2013, 7, 1); // Coba bulan Juli 2013
var CHIRPS_monthly_test_raw = CHIRPS.filterBounds(lembang2).filterDate(testDate, testDate.advance(1, 'month'));
var monthly_precipitation_test = CHIRPS_monthly_test_raw.select('precipitation').sum().clip(lembang2);

// Pastikan ada band sebelum visualisasi
var hasTestCHIRPSData = monthly_precipitation_test.bandNames().contains('precipitation');

if (hasTestCHIRPSData) {
  var CHIRPSParams = {min: 0, max: 500, palette: ['white', 'blue', 'darkblue']}; // Curah hujan dalam mm
  Map.addLayer(monthly_precipitation_test, CHIRPSParams, 'CHIRPS Test Image ' + testDate.format('YYYY-MM'));
} else {
  print('No valid data for CHIRPS test image for ' + testDate.format('YYYY-MM'));
}

Map.addLayer(lembang2, {}, 'Lembang Boundary');
Map.centerObject(lembang2, 10);
*/

// Print total sampel yang dihasilkan di konsol
print('Total samples for all months (CHIRPS only):', all_sampled_CHIRPS_data.size());